In [ ]:
import os
from zipfile import ZipFile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tsfel
from datetime import datetime

import sys
sys.path.append('../')
from vibrodiagnostics import (
    datasets,
    discovery,
    models
)

TEMPORAL_NAME = 'industry_temporal.csv'
SPECTRAL_NAME = 'industry_spectral.csv'
FEATURES_PATH =  '../../datasets/features_data/'

Fs = 26866      # 26667
DATASET_PATH = '../../datasets/Pumps-and-Compressors/'

In [ ]:
def features_produce(domain, dataframe, metadata):
    result = []
    for i, df in enumerate(dataframe):
        fvector = metadata
        for col in ['x', 'y', 'z']:
            if domain == 'temporal':
                v = discovery.time_features_calc(df, col)
            elif domain == 'spectral':
                v = discovery.frequency_features_calc(df, col, 2**14, Fs)
            fvector.extend(v)
        result.append(pd.DataFrame(dict(fvector))) 
    return result


def features_calculate(dataset, domain):
    feat = []
    for name, ts in dataset:
        header = name.split(os.path.sep)
        metadata = [
            ('place', header[-5]),
            ('date', datetime.fromisoformat(header[-4]).date()),
            ('device', header[-3]),
            ('position', header[-2]),
            ('seq', int(header[-1].split('.')[0]))
        ]
        dataframe = discovery.split_dataframe(ts, parts=60//5)
        feat.extend(features_produce(domain, dataframe, metadata.copy()))

    return pd.concat(feat).reset_index(drop=True)

Export features for all files

In [ ]:
list_of_files = datasets.steval_files(DATASET_PATH)
DATASET = datasets.steval_dataset(list_of_files, Fs)

In [ ]:
FEATURES_FILENAME = os.path.join(FEATURES_PATH, TEMPORAL_NAME)

features = features_calculate(DATASET, 'temporal')
features.to_csv(FEATURES_FILENAME, index=False)
features

In [ ]:
FEATURES_FILENAME = os.path.join(FEATURES_PATH, SPECTRAL_NAME)

features = features_calculate(DATASET, 'spectral')
features.to_csv(FEATURES_FILENAME, index=False)
features